### Analyzing SSPs with Object-Specific Axes

Ideally, one would like to encode a variety of objects in a single SSP and apply a single transformation to the SSP to advance the position of each object along its own unique trajectory. Earlier work illustrates that it is straightforward to apply a linear transformation that advances the position of a single object in isolation, or a linear transformation that advances the position of all objects identically. 

Below, the use of different spatial axis vectors to encode each object is considered as a solution to this difficulty.

In [ ]:
%matplotlib inline

In [ ]:
import base64
import itertools

from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from ssp.maps import Spatial2D

In [ ]:
dim = 256
scale = 10

T = 2
dt = 0.05

x_len = 2
y_len = 2
x_spaces = 101
y_spaces = 101

#### 1. Object-Specific Spatial Axes

If a set of objects are represented as 

$M=X^{x_1}\circledast Y^{y_1} + X^{x_2}\circledast Y^{y_2}...$

then it is possible to shift all objects as follows:

$M=M \circledast X^{\Delta x} \circledast Y^{\Delta y}$

due to the properties of fractional exponentiation with HRRs. It is not easy, however, to shift each object independently using this approach. So, we might introduce separate axis encoding vectors for each object and have different parts of a transformation act on specific subsets of these vectors. For instance, we could encode objects as follows:

$M=X_A^{x_1}\circledast Y_A^{y_1} + X_B^{x_2}\circledast Y_B^{y_2}...$

and then do the follow to apply separate transformations to each object:

$M=M \circledast (X_A^{\Delta x_1} \circledast Y_A^{\Delta y_1} + X_B^{\Delta x_2} \circledast Y_B^{\Delta y_2})$

This would produce an updated $M$ with $N^2 - N$ noise terms if $N$ is the number of encoded objects, since each term in the bracketed sum would apply to a corresponding term in $M$ and combine with the other $N-1$ terms in $M$ to yield noise. There are $N$ terms in the bracketed sum, so the creation of $N-1$ noise terms occurs $N$ times, leading to a total of $N^2 - N$ noise terms overall. This is obviously not great for scaling up to large numbers of objects, even if the noise is zero-mean. Below is an illustration of the effect of this scaling:

In [ ]:
# create vocab keys for object-specific axis vectors
axes = ['X', 'Y']
objs = ['A', 'B', 'C']
keys = [x+y for x, y in itertools.product(axes, objs)]

# create default spatial grid and add vectors for new keys
ssp_map = Spatial2D(dim=dim, scale=scale)
ssp_map.build_grid(x_len, y_len, x_spaces, y_spaces)
ssp_map.voc.populate(";".join([k + '.unitary()' for k in keys]))

# create axis-specific encoding function based on object name
def encode(name, x, y):
    x = ssp_map.voc['X' + name] ** (x * scale)
    y = ssp_map.voc['Y' + name] ** (y * scale)
    return x * y

# create grids for each pair of axis keys for object-specific heatmaps
grids = {obj: np.zeros_like(ssp_map.ssp_tensor) for obj in objs}

for name, grid in grids.items():
    for i, x in enumerate(ssp_map.xs):
        for j, y in enumerate(ssp_map.ys):
            grid[-j, i] = encode(name, x, y).v

# create a sample SSP encoding three objects
ssp = encode('A', 1, 1) + encode('B', 0.25, 0.5) + encode('C', 1.5, 1)

In [ ]:
# create separate heatmap for each encoded object location
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)

def plot_setup(figsize, n_subplots):
    fig = plt.figure(figsize=figsize)
    fig_axes = []
    
    for n in range(1, n_subplots + 1):
        ax = fig.add_subplot(1, n_subplots, n)
        fig_axes.append(ax)
    
    return fig, fig_axes



fig, fig_axes = plot_setup(figsize=(16, 5), n_subplots=3)

for name, ax in zip(objs, fig_axes):
    sim = np.tensordot(grids[name], ssp.v, axes=[[2], [0]])
    ax.imshow(sim, vmin=-1, vmax=1, cmap=cmap)
    ax.set_title("Object %s" % name)
    ax.set_xticks([])
    ax.set_yticks([])

plt.show()

In [ ]:
# now apply a transformation to the SSP to shift each object
Adx = 0.3
Ady = 0.3

Bdx = -0.1
Bdy = -0.2

Cdx = -0.5
Cdy = 0.3

W = encode('A', Adx, Ady) + encode('B', Bdx, Bdy) + encode('C', Cdx, Cdy)
ssp = ssp * W

fig, fig_axes = plot_setup(figsize=(16, 5), n_subplots=3)

for name, ax in zip(objs, fig_axes):
    sim = np.tensordot(grids[name], ssp.v, axes=[[2], [0]])
    ax.imshow(sim, vmin=-1, vmax=1, cmap=cmap)
    ax.set_title("Object %s" % name)
    ax.set_xticks([])
    ax.set_yticks([])

plt.show()

In [ ]:
# now create an animation that applies the transform W repeatedly
fig, fig_axes = plot_setup(figsize=(16, 5), n_subplots=3)

images = []

for t in range(int(T / dt)):
    current_frame = []
    for name, ax in zip(objs, fig_axes):
        sim = np.tensordot(grids[name], ssp.v, axes=[[2], [0]])
        im = ax.imshow(sim, vmin=-1, vmax=1, cmap=cmap, animated=True)
        current_frame.append(im)
        
    # apply transformation before moving to next timestep
    ssp = ssp * W
    images.append(current_frame)
        

for ax in fig_axes:
    ax.set_xticks([])
    ax.set_yticks([])    
    ax.set_title("Object %s" % name)


ani = animation.ArtistAnimation(fig, images, interval=1, blit=True)

plt.close()

fname = ".atemp.gif"
ani.save(fname, writer='imagemagick')
gif = open(fname, "rb").read()
gif_base64 = base64.b64encode(gif).decode()
HTML('<img src="data:image/gif;base64,{0}" />'.format(gif_base64))

Overall, this method is even worse than initially assumed, since the noise terms compound at each time step, quickly pushing the signal-to-noise ratio towards zero. Clearly, a cleanup process of some kind is needed between subsequent time-steps in for an approach involving object-specific axis representations to scale effectively. 

Formally, we can describe the growth of noise terms over time as follows: each term in the bracketed sum described above applies to one corresponding term in $M$ and combines with the other $N-1$ terms in $M$ to yield noise. There are $N$ terms in the bracketed sum, so the creation of $N−1$ noise terms occurs $N$ times, leading to a total of $N^2-N$ noise terms after a single time step. On the next time step, there each term in the bracketed sum applies to one corresponding term in $N$, and then combines with the $N-1$ other encoding terms and the $N^2 - N$ noise terms created by on the previous time step; again, these combinations $N$ times. This yields a total of $N * (N-1 + N^2 - N) = N^3 - N$ noise terms on the second time step. After $t$ timesteps, the number of noise terms is $N^{t+1} - N$.

In [ ]:
# plot N / (N^(t+1) - N) to show noise degradation as a function of time
timesteps = 10
n_values = np.arange(1, 6)

def snr(n, timesteps):
    times = np.arange(0, timesteps)
    noise = [n**(t+1) - n for t in times]
    return np.array([n / x if x > 0 else n for x in noise])  # avoid divide by zero

plt.figure()
for n in n_values:
    vals = snr(n, timesteps)
    plt.plot(np.arange(0, 10), vals)
    
plt.legend(n_values)
plt.xlabel("Timesteps")
plt.ylabel("Signal Terms / Noise Terms")
plt.title("SNR for N=1 to N=5 encodings")
plt.show()